In [4]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import pygame
import threading

# Initialize pygame mixer for sound
pygame.mixer.init()
alarm_sound = pygame.mixer.Sound("beep.mp3")

# Initialize video capture and dlib detector
cap = cv2.VideoCapture(1)
detector = dlib.get_frontal_face_detector()

# Load the pre-trained model for facial landmarks
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# State tracking variables
sleep = drowsy = active = 0
status = ""
color = (0, 0, 0)
alarm_playing = False

# Function to compute Euclidean distance
def compute(ptA, ptB):
    return np.linalg.norm(ptA - ptB)

# Function to determine blink state
def blinked(p1, p2, p3, p4, p5, p6):
    up = compute(p2, p6) + compute(p3, p5)
    down = compute(p1, p4)
    ratio = up / (2.0 * down)
    if ratio > 0.26:
        return 2  # Eyes open
    elif 0.21 < ratio <= 0.26:
        return 1  # Drowsy
    else:
        return 0  # Eyes closed

# Function to play alarm sound
def play_alarm():
    global alarm_playing
    if not alarm_playing:
        alarm_playing = True
        alarm_sound.play()
        pygame.time.wait(int(alarm_sound.get_length() * 1000))  # Wait for the sound to complete
        alarm_playing = False

# Main processing loop
if cap.isOpened():
    print("Starting video capture...")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame. Exiting...")
            break

        # Resize frame for consistent processing speed
        frame = cv2.resize(frame, (640, 480))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = detector(gray)
        for face in faces:
            x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Detect facial landmarks
            landmarks = predictor(gray, face)
            landmarks = face_utils.shape_to_np(landmarks)

            # Detect eye states
            left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], 
                                 landmarks[39], landmarks[40], landmarks[41])
            right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], 
                                  landmarks[45], landmarks[46], landmarks[47])

            # Update states
            if left_blink == 0 or right_blink == 0:
                sleep += 1
                drowsy = active = 0
                if sleep > 6:
                    status = "SLEEPING !!!"
                    color = (0, 0, 255)
                    # Trigger alarm sound in a separate thread
                    threading.Thread(target=play_alarm).start()
            elif left_blink == 1 or right_blink == 1:
                sleep = active = 0
                drowsy += 1
                if drowsy > 6:
                    status = "Drowsy !"
                    color = (255, 0, 0)
            else:
                sleep = drowsy = 0
                active += 1
                if active > 6:
                    status = "Active :)"
                    color = (0, 255, 0)

            # Display the status on the frame
            cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

            # Draw facial landmarks
            for (x, y) in landmarks:
                cv2.circle(frame, (x, y), 1, (255, 255, 0), -1)

        # Display the processed frame
        cv2.imshow("Drowsiness Detection", frame)

        # Exit on pressing ESC
        key = cv2.waitKey(1)
        if key == 27:  # ESC key
            print("Exiting...")
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
else:
    print("Error: Camera not accessible.")


Starting video capture...
Exiting...
